In [15]:

import requests
import json
import math
from nsepython import * 

url_oc      = "https://www.nseindia.com/option-chain"
url_eq = "https://www.nseindia.com/api/option-chain-equities?symbol="

# Headers
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36',
            'accept-language': 'en,gu;q=0.9,hi;q=0.8',
            'accept-encoding': 'gzip, deflate, br'}

sess = requests.Session()
cookies = dict()

# Local methods
def set_cookie():
    request = sess.get(url_oc, headers=headers, timeout=5)
    cookies = dict(request.cookies)
    
def get_data(url):
    set_cookie()
    payload = nsefetch(url)
    return payload
    
    # response = sess.get(url_oc, headers=headers, timeout=5, cookies=cookies)
    # set_cookie()
    # response = sess.get(url, headers=headers, timeout=5, cookies=cookies)
    # if(response.status_code==200):
    #     return response.text
    # return ""


def getOIInterPretation(lp_change, coi):
    oi_intrepretation = ""
    if(lp_change>0 and coi>0):
        oi_intrepretation = "Long Buildup"
    if(lp_change>0 and coi<0):
        oi_intrepretation = "Short covering"
    if(lp_change<0 and coi<0):
       oi_intrepretation = "Long Liquidation"
    if(lp_change<0 and coi>0):
       oi_intrepretation = "Short Buildup"
    return oi_intrepretation

def print_oi(num,url):
    data = get_data(url)
    # data = json.loads(response_text)
    step = data['records']['strikePrices'][1] - data['records']['strikePrices'][0]
    option_chain_data = data['records']['data']
    ltp = option_chain_data[0]['PE']['underlyingValue']
    strike_price_list = [x['strikePrice'] for x in option_chain_data]
    atm_strike = sorted([[round(abs(ltp-i),2),i] for i in strike_price_list])[0][1]
    nearest = atm_strike
    strike = nearest - (step*num)
    start_strike = nearest - (step*num)
    currExpiryDate = data["records"]["expiryDates"][0]
    
    oi_interpreted = []
    for item in data['records']['data']:
        if item["expiryDate"] == currExpiryDate:
            if item["strikePrice"] == strike and item["strikePrice"] < start_strike+(step*num*2):
                obj = {}
                
                obj["ce_oi"] = int(item["CE"]["openInterest"])
                obj["ce_coi"] = float(item["CE"]["changeinOpenInterest"])
                obj["ce_lp"] = float(item["CE"]["lastPrice"])
                obj["ce_lp_change"] = float(item["CE"]["change"])
                obj["ce_oi_inteerpretation"] = getOIInterPretation(obj["ce_lp_change"], obj["ce_coi"])
                obj["strikePrice"] = str(item["strikePrice"])
                obj["pe_lp_change"] = float(item["PE"]["change"])
                obj["pe_oi"] = int(item["PE"]["openInterest"])
                obj["pe_coi"] = float(item["PE"]["changeinOpenInterest"])
                obj["pe_lp"] = float(item["PE"]["lastPrice"])
                obj["pe_oi_inteerpretation"] = getOIInterPretation(obj["pe_lp_change"], obj["pe_coi"])
                oi_interpreted.append(obj)
                # print((str(item["strikePrice"])) + (" CE ") + "[ " + (str(item["CE"]["openInterest"]).rjust(30," ")) + " " + obj["ce_oi_inteerpretation"] + " ]" + 
                                                #    (" PE ") + "[ " + (str(item["PE"]["openInterest"]).rjust(30," ")) + " " + obj["pe_oi_inteerpretation"] + " ]")
                strike = strike + step
    return oi_interpreted

In [22]:
oi_interpreted = print_oi(10, url_eq + "TCS")
fdf = pd.DataFrame(oi_interpreted)
print(fdf[["ce_oi_inteerpretation", "ce_lp","ce_lp_change", "strikePrice", "pe_lp_change", "pe_lp","pe_oi_inteerpretation"]])

   ce_oi_inteerpretation   ce_lp  ce_lp_change strikePrice  pe_lp_change  \
0                           0.00          0.00        3120         -0.50   
1       Long Liquidation  192.70         -9.30        3140         -0.60   
2       Long Liquidation  169.30        -15.55        3160         -0.70   
3       Long Liquidation  150.00        -20.00        3180         -0.65   
4       Long Liquidation  133.95        -16.45        3200         -0.60   
5       Long Liquidation  110.80        -24.20        3220         -0.45   
6       Long Liquidation   96.35        -18.30        3240         -0.20   
7       Long Liquidation   81.35        -15.80        3260          0.75   
8       Long Liquidation   67.25        -13.35        3280          1.35   
9       Long Liquidation   53.90        -14.15        3300          3.05   
10         Short Buildup   42.35        -13.10        3320          4.25   
11         Short Buildup   32.75        -11.20        3340          5.60   
12         S

In [24]:
sum_coi_ce = fdf["ce_coi"].sum()
sum_coi_pe = fdf["pe_coi"].sum()

sum_lpc_ce = fdf["ce_lp_change"].sum()
sum_lpc_pe = fdf["pe_lp_change"].sum()

def getIntraDayTrend(sum_coi_ce, sum_coi_pe):
    if(sum_coi_ce > sum_coi_pe):
        return "BEARISH"
    elif (sum_coi_ce < sum_coi_pe):
        return "BULLISH"
    
intra_day_trend = getIntraDayTrend(sum_coi_ce, sum_coi_pe)
overall_trend = getIntraDayTrend(sum_lpc_pe, sum_lpc_ce)

print(intra_day_trend)
print(overall_trend)

BEARISH
BEARISH


In [27]:
if overall_trend == "BEARISH":
    print("BUY PE")
elif (overall_trend == "BULLISH"):
    print("Buy CE")

BUY PE
